In [28]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# Define custom dataset
class PlantDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_id = str(self.data_frame.iloc[idx, 0])
        img_path = os.path.join(self.root_dir, img_id + '.jpeg')
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        ancillary_data = self.data_frame.iloc[idx, 1:164].values.astype('float32')
        if 'train' in self.root_dir:
            labels = self.data_frame.iloc[idx, 164:].values.astype('float32')
            return [ancillary_data, image], labels
        else:
            return [ancillary_data, image], img_id

# Define transformations
data_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    # No noise needed since clear images
])

# Load the datasets
train_dataset = PlantDataset(csv_file='data/train.csv', root_dir='data/train_images', transform=data_transforms)
test_dataset = PlantDataset(csv_file='data/test.csv', root_dir='data/test_images', transform=data_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [29]:
print(len(train_dataset))
print(len(test_dataset))
print(len(train_loader))
print(len(test_loader))

43363
6391
1356
200


In [30]:
# Define the CNN model for regression
class CNNRegressionModel(nn.Module):
    def __init__(self):
        super(CNNRegressionModel, self).__init__()
        self.cnn = models.resnet50(pretrained=True)
        # print(self.cnn.fc.in_features)
        self.fc = nn.Sequential(
            nn.Linear(self.cnn.fc.in_features + 163, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 6)
        )
        self.cnn.fc = nn.Identity()  # Remove the last fully connected layer

    def forward(self, x):
        ancillary_data, images = x
        cnn_out = self.cnn(images)
        combined = torch.cat((cnn_out, ancillary_data), dim=1)
        out = self.fc(combined)
        return out

In [31]:
# Instantiate the models, loss function, and optimizers
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = CNNRegressionModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Using device: cuda


c:\Users\YZM\Desktop\for fun\pytorch\torch-playground\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\YZM\Desktop\for fun\pytorch\torch-playground\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


2048


In [45]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        ancillary_data, images = inputs
        ancillary_data, images, labels = ancillary_data.to(device), images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model([ancillary_data, images])
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}')


Epoch 1/10, Loss: 566753.0010
Epoch 2/10, Loss: 556726.1136
Epoch 3/10, Loss: 517756.4652
Epoch 4/10, Loss: 444312.1582
Epoch 5/10, Loss: 513669.3934
Epoch 6/10, Loss: 408333.1517
Epoch 7/10, Loss: 426217.3737
Epoch 8/10, Loss: 397726.1440
Epoch 9/10, Loss: 509838.8231
Epoch 10/10, Loss: 392908.7052


In [47]:
torch.save(model.state_dict(), 'CNNregression.pth')

print("Models saved successfully.")

Models saved successfully.


In [46]:
# Prediction and Submission
model.eval()
predictions = []
image_ids = []
with torch.no_grad():
    for inputs, ids in test_loader:
        ancillary_data, images = inputs
        ancillary_data, images = ancillary_data.to(device), images.to(device)
        outputs = model([ancillary_data, images])
        predictions.extend(outputs.cpu().numpy())
        image_ids.extend([int(id) for id in ids])

# Prepare the submission file
submission = pd.DataFrame(predictions, columns=['X4', 'X11', 'X18', 'X26', 'X50', 'X3112'])
submission['id'] = image_ids
submission = submission[['id', 'X4', 'X11', 'X18', 'X26', 'X50', 'X3112']]
submission.to_csv('j228ye_cnnregression.csv', index=False)